In [1]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from animal_shelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name

username = "aacuser"
password = "password"

# Connect to database via CRUD Module
animals = AnimalShelter(username, password)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(animals.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'Grazioso_Salvare_Logo.png' 
encoded_image = base64.b64encode(open(image_filename, 'rb').read())


app.layout = html.Div([
    html.Center(html.B(html.H1('Shaelyn Turner Project Two'))),
    html.Hr(),
    html.Img(
        src='data:image/png;base64,{}'.format(encoded_image.decode()),
        style={'height':'20%','width':'20%'}
        ),
    html.Div(
        
        # Code for the interactive filtering options (radio buttons)
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'Reset', 'value': 'Reset'},
                {'label': 'Water Rescue', 'value': 'Water'},
                {'label': 'Mountain or Wilderness Rescue', 'value': 'Mountain'},
                {'label': 'Disaster or Individual Tracking', 'value': 'Disaster'}
            ]
        )

    ),
     html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable="single",
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current= 0,
        page_size= 10
        
        
    ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
        html.Div(className='row',
         style={'display' : 'flex'},
             children=[
                html.Div([
                # Dropdown menu for pie chart
                    dcc.Dropdown(
                        id='pie_dropdown',
                        className='col s12 m6',
                        # options are Age and Breed
                        options=[
                            {'label': 'Breed', 'value': 'breed'},
                            {'label': 'Age', 'value': 'age_upon_outcome'},       
                        ],
                        value='breed',
                        multi=False,
                        clearable=False
                        )
                    ],
                style={"width":"20%"}),
                
                html.Div(
                    id='graph-id',
                    className='col s12 m6',
                ),
                
                html.Div(
                    id='map-id',
                    className='col s12 m6',
                    )])
])


#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback([Output('datatable-id','data'),
              Output('datatable-id','columns')],
              [Input('filter-type', 'value')])
## FIX ME Add code to filter interactive data table with MongoDB queries              
def update_dashboard(filter_type):
    # Reste the filters
    if filter_type == 'Reset':
        df = pd.DataFrame.from_records(animals.read({})) 
    
    # Water Filter
    elif filter_type == 'Water':
        df = pd.DataFrame(list(animals.read({
            "animal_type" : "Dog", 
            "breed" : {"$in":["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]},
            "sex_upon_outcome" : "Intact Female", 
            "age_upon_outcome_in_weeks" : {"$gte" : 26}, 
            "age_upon_outcome_in_weeks" : {"$lte" : 156},
            })))
    # Mountain and Wliderness Filter                     
    elif filter_type == 'Mountain': 
        df = pd.DataFrame(list(animals.read({
            "animal_type" : "Dog", 
            "breed" : {"$in":["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", 
                          "Siberian Husky", "Rottweiler"]},
            "sex_upon_outcome" : "Intact Male", 
            "age_upon_outcome_in_weeks" : {"$gte" : 26}, 
            "age_upon_outcome_in_weeks" : {"$lte" : 156},
            })))
        
    # Disaster and Inidividual Tracking Fliter
    elif filter_type == 'Disaster': 
        df = pd.DataFrame(list(animals.read({
            "animal_type" : "Dog", 
            "breed" : {"$in":["Doberman Pinscher", "German Shepherd", "Golden Retriever", 
                          "Bloodhound", "Rottweiler"]},
            "sex_upon_outcome" : "Intact Male", 
            "age_upon_outcome_in_weeks" : {"$gte" : 20}, 
            "age_upon_outcome_in_weeks" : {"$lte" : 300},
            })))
    else: 
        df = pd.DataFrame.from_records(animals.read({})),
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
        data=df.to_dict('records')
        selected_rows = [0]
        return (data, columns)



# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data"),
     Input('pie_dropdown', 'value')])
def update_graphs(viewData, dropdownValue):
    
    ###FIX ME ####
    # add code for chart of your choice (e.g. pie chart) #
    dff = pd.DataFrame.from_dict(viewData)
    return [
        dcc.Graph(            
            figure = px.pie(
                data_frame=dff, 
                names=dropdownValue)

    #return [
        #dcc.Graph(            
#            figure = px.pie(df, names='breed', title='Preferred Animals')
            #figure = px.histogram(dff, x='breed')
        )    
    ]
    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'derived_virtual_selected_rows')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])

    
    
#FIXME Add in the code for your geolocation chart
def update_map(viewData, selected_rows):
    #default center and zoom
    center = [20,0]
    zoom = 2
    
    if viewData is None or not selected_rows:
        # Return Default if No Row is Selected
        return [dl.Map(center=center, zoom=zoom, children=[dl.TileLayer()], style={'width':'1000px', 'height':'500px'})] 

    rows = pd.DataFrame.from_records(viewData)
    
    if selected_rows and viewData and selected_rows[0] < len(rows):
        
        #If Row is Selected Row
        selected_row = rows.iloc[selected_rows[0]] 
        latitude = selected_row['location_lat']
        longitude = selected_row['location_long']
        
        # Center On Selected Row
        center = [latitude, longitude] 
        
        # Zoom In to Location
        zoom = 10 
        
        # Create a Marker
        marker = dl.Marker(position=center, children=[
            dl.Tooltip("Selected Location"),
            dl.Popup("This is the selected location.")
        ])
        
        #Updating the Map's Children to Include the New Marker
        return [dl.Map(center=center, zoom=zoom, children=[
            dl.TileLayer(),
            marker
        ], style={'width': '1000px', 'height': '500px'})]


app.run_server(debug=True)


Dash app running on http://127.0.0.1:18508/
